# Exploratory Data Analysis

In [13]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import math
from datetime import datetime, date, time

In [14]:
data_folder='../data/'
df_t = pd.read_csv(data_folder + '2t.csv')
print(df_t.shape)
pd.set_option('display.max_columns', None)
df_t.head()

(68300, 3)


,date,state_name,2m max temperature
0,2005-01-01,ILLINOIS,281.98935
1,2005-01-02,ILLINOIS,287.57990
2,2005-01-03,ILLINOIS,280.19630
3,2005-01-04,ILLINOIS,277.75946
4,2005-01-05,ILLINOIS,275.99216


In [15]:
data_folder='../data/'
df_p = pd.read_csv(data_folder + 'tp.csv')
print(df_p.shape)
pd.set_option('display.max_columns', None)
df_p.head()

(68330, 3)


,date,state_name,total precipitation
0,2005-01-01,ILLINOIS,2.398395
1,2005-01-02,ILLINOIS,7.677360
2,2005-01-03,ILLINOIS,24.067541
3,2005-01-04,ILLINOIS,5.230138
4,2005-01-05,ILLINOIS,27.008377


In [16]:
data_folder='../data/'
df_yield = pd.read_csv(data_folder + 'yield.csv')
print(df_yield.shape)
pd.set_option('display.max_columns', None)
df_yield.head()

(554, 3)


,state_name,year,yield
0,ALABAMA,2005,33.0
1,ARKANSAS,2005,34.0
2,DELAWARE,2005,26.0
3,FLORIDA,2005,32.0
4,GEORGIA,2005,26.0


## Data Processing

In [17]:
df_t.rename(columns={"state_name": "state", "2m max temperature": "temp_K"}, inplace=True)
df_p.rename(columns={"state_name": "state", "total precipitation": "precip_mm"}, inplace=True)

In [18]:
# Convert kelvin to celsius
df_t.iloc[:, 2:] = df_t.iloc[:, 2:] - 273.15
df_t.rename(columns={"temp_K": "temp_C"}, inplace=True)
df_t.head()


,date,state,temp_C
0,2005-01-01,ILLINOIS,8.83935
1,2005-01-02,ILLINOIS,14.42990
2,2005-01-03,ILLINOIS,7.04630
3,2005-01-04,ILLINOIS,4.60946
4,2005-01-05,ILLINOIS,2.84216


In [19]:
# reshape date
df_t["date"] = pd.to_datetime(df_t["date"])
df_t["year"] = df_t["date"].dt.year
df_t["month"] = df_t["date"].dt.month

df_p["date"] = pd.to_datetime(df_p["date"])
df_p["year"] = df_p["date"].dt.year
df_p["month"] = df_p["date"].dt.month

#filter by growing season (ie. May–September)
grow_months = [5, 6, 7, 8, 9]
df_t_grow = df_t[df_t["month"].isin(grow_months)]
df_p_grow = df_p[df_p["month"].isin(grow_months)]

# Merge dataframes
t_features = df_t_grow.groupby(["state", "year"]).agg(
    avg_temp_C=("temp_C", "mean"),
    max_temp_C=("temp_C", "max"),
    min_temp_C=("temp_C", "min"),
).reset_index()

p_features = df_p_grow.groupby(["state", "year"]).agg(
    total_precip_mm=("precip_mm", "sum"),
    avg_precip_mm=("precip_mm", "mean")
).reset_index()

df_yield.rename(columns={"state_name": "state"}, inplace=True)  
full_df = df_yield.merge(t_features, on=["state", "year"]).merge(p_features, on=["state", "year"])
full_df.head()

# Check for missing values
missing_values = full_df.isnull().sum()
print("Missing values in each column:", missing_values)



Missing values in each column: state              0
year               0
yield              0
avg_temp_C         0
max_temp_C         0
min_temp_C         0
total_precip_mm    0
avg_precip_mm      0
dtype: int64


## Add extrem event

In [20]:
# Hot days > 35°C
hot_days = df_t_grow[df_t_grow["temp_C"] > 35] \
    .groupby(["state", "year"]).size().reset_index(name="hot_days")

# Freezing days < 5°C
freezing_days = df_t_grow[df_t_grow["temp_C"] < 5] \
    .groupby(["state", "year"]).size().reset_index(name="freezing_days")

# Dry days < 1 mm
dry_days = df_p_grow[df_p_grow["precip_mm"] < 1] \
    .groupby(["state", "year"]).size().reset_index(name="dry_days")

# Flooding days > 20 mm
flooding_days = df_p_grow[df_p_grow["precip_mm"] > 20] \
    .groupby(["state", "year"]).size().reset_index(name="flooding_days")
    
extreme_features = hot_days \
    .merge(freezing_days, on=["state", "year"], how="outer") \
    .merge(dry_days, on=["state", "year"], how="outer") \
    .merge(flooding_days, on=["state", "year"], how="outer")

# Fill NaNs with 0 (no extreme events observed in that year)
extreme_features.fillna(0, inplace=True)

# Then merge with your main feature set
full_df2 = full_df.merge(extreme_features, on=["state", "year"], how="left")
full_df2.head()


,state,year,yield,avg_temp_C,max_temp_C,min_temp_C,total_precip_mm,avg_precip_mm,hot_days,freezing_days,dry_days,flooding_days
0,ILLINOIS,2005,46.5,28.029765,36.79556,10.09295,351.391859,2.296679,2.0,0.0,88,0.0
1,INDIANA,2005,49.0,27.139371,34.91238,10.42950,419.308881,2.740581,0.0,0.0,90,2.0
2,IOWA,2005,52.5,26.673741,35.29424,8.72470,400.706764,2.618998,1.0,0.0,92,2.0
3,KANSAS,2005,37.0,29.811761,37.96978,12.98600,390.994898,2.555522,16.0,0.0,88,3.0
4,MINNESOTA,2005,45.5,24.188087,32.77953,6.20060,460.882343,3.012303,0.0,0.0,72,1.0


In [21]:
# No freeze days in the dataset - remove this column
full_df2.drop(columns=["freezing_days"], inplace=True)
full_df2

# to csv
full_df2.to_csv('../data/df_processed.csv', index=False)